In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
import torch
from transformers import Trainer, TrainingArguments

In [3]:
cyberbullying_tweets = pd.read_csv('../cleaned/cyberbullying_tweets_cleaned.csv')

In [4]:
cyberbullying_tweets.head()

,tweet_text,cyberbullying_type,cleaned_text
0,"In other words #katandandre, your food was cra...",not_cyberbullying,in other words katandandre your food was crapi...
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,why is aussietv so white mkr theblock imaceleb...
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,a classy whore or more red velvet cupcakes
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,meh p thanks for the heads up but not too con...
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,this is an isis account pretending to be a kur...


In [4]:
label_map = {
    'age': 0,
    'gender': 1,
    'religion': 2,
    'ethnicity': 3,
    'not_cyberbullying': 4,
    'other_cyberbullying': 5
}


cyberbullying_tweets['label'] = cyberbullying_tweets['cyberbullying_type'].map(label_map)

In [5]:
print(cyberbullying_tweets['label'].value_counts())  # check class distribution

label
2    7998
1    7973
4    7945
5    7823
0    6839
Name: count, dtype: int64


In [6]:
# Split the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    cyberbullying_tweets['cleaned_text'].tolist(), cyberbullying_tweets['label'].tolist(), test_size=0.2, random_state=42
)

In [7]:
# Ensure all tweet_text entries are strings and drop any NaNs
train_texts = [str(text) for text in train_texts if pd.notnull(text)]
val_texts = [str(text) for text in val_texts if pd.notnull(text)]

In [8]:
# Trim the longer one to match the shorter one
min_len_train = min(len(train_texts), len(train_labels))
train_texts = train_texts[:min_len_train]
train_labels = train_labels[:min_len_train]

In [9]:
min_len_val = min(len(val_texts), len(val_labels))
val_texts = val_texts[:min_len_val]
val_labels = val_labels[:min_len_val]

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
class CyberbullyingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

train_dataset = CyberbullyingDataset(train_encodings, train_labels)
val_dataset = CyberbullyingDataset(val_encodings, val_labels)

In [12]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
training_args = TrainingArguments(
    output_dir='./cyberbullying_results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    eval_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [14]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jkos0012 (jkos0012-monash-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.613200,1.611953
2,1.610300,1.608887
3,1.604100,1.609021
4,1.598200,1.613477


TrainOutput(global_step=7672, training_loss=1.6129823065400746, metrics={'train_runtime': 3214.7288, 'train_samples_per_second': 38.172, 'train_steps_per_second': 2.387, 'total_flos': 8072010847236096.0, 'train_loss': 1.6129823065400746, 'epoch': 4.0})

In [ ]:
trainer.evaluate()

In [15]:
# Save
model.save_pretrained("bert-cyberbullying")
tokenizer.save_pretrained("bert-cyberbullying")

('bert-cyberbullying/tokenizer_config.json',
 'bert-cyberbullying/special_tokens_map.json',
 'bert-cyberbullying/vocab.txt',
 'bert-cyberbullying/added_tokens.json')

In [17]:
!zip -r bert-cyberbullying.zip bert-cyberbullying

  adding: bert-cyberbullying/ (stored 0%)
  adding: bert-cyberbullying/config.json (deflated 54%)
  adding: bert-cyberbullying/vocab.txt (deflated 53%)
  adding: bert-cyberbullying/special_tokens_map.json (deflated 42%)
  adding: bert-cyberbullying/tokenizer_config.json (deflated 75%)
  adding: bert-cyberbullying/model.safetensors (deflated 7%)


In [18]:
from google.colab import files
files.download('bert-cyberbullying.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>